<a href="https://colab.research.google.com/github/CarlosLeandro09/DataAnalysisRadiology/blob/main/Patient_Treatment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I don't remenber if it's mine rsrs

# **Initial observations:**

*   This notebook is focused on providing *tools* for building and avaluating models.
*   Relative to the model applied to the worked dataset there is *no great concern with accuracy*.

# **1. Let's import the relevant packages**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score,roc_auc_score

from statistics import mean
import pickle

# **2. We have also import our dataset and spend time analyzing**

This dataset was downloaded from Mendeley Data.

Sadikin, Mujiono (2020), “EHR Dataset for Patient Treatment Classification”, Mendeley Data, V1, doi: 10.17632/7kv3rctx7m.1

About: "The dataset is Electronic Health Record Predicting collected from a private Hospital in Indonesia.It contains the patient's laboratory test results used to determine next patient treatment whether in care or out care."

Source: https://www.kaggle.com/manishkc06/patient-treatment-classification

In [ ]:
data = pd.read_csv('Patient_Treatment_Classification.csv')
data.head()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX,SOURCE
0,33.8,11.1,4.18,4.6,150,26.6,32.8,80.9,33,F,1
1,44.6,14.0,6.86,6.3,232,20.4,31.4,65.0,36,M,0
2,42.9,14.0,4.57,6.2,336,30.6,32.6,93.9,70,F,0
3,41.9,14.4,4.67,3.5,276,30.8,34.4,89.7,18,F,0
4,40.6,13.3,4.85,14.9,711,27.4,32.8,83.7,36,M,0


In this section we must do the statistical analysis of the dataset.

Note that there are children and adults.

In [ ]:
data.describe()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SOURCE
count,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000,3309.000000
mean,38.226111,12.749350,4.544802,8.715533,258.893019,28.230039,33.336476,84.611333,46.644303,0.398005
std,5.971943,2.084325,0.784510,4.991299,112.676139,2.696520,1.247055,6.916079,21.874106,0.489561
min,13.700000,3.800000,1.480000,1.100000,10.000000,14.900000,26.000000,54.000000,1.000000,0.000000
25%,34.300000,11.400000,4.040000,5.700000,191.000000,27.200000,32.700000,81.500000,29.000000,0.000000
50%,38.700000,12.900000,4.580000,7.600000,257.000000,28.700000,33.400000,85.300000,48.000000,0.000000
75%,42.500000,14.200000,5.060000,10.300000,322.000000,29.800000,34.100000,88.800000,64.000000,1.000000
max,69.000000,18.900000,7.860000,76.600000,1121.000000,40.800000,38.400000,115.600000,99.000000,1.000000


It is important to note the format of each column for later need.

In [ ]:
data.dtypes

HAEMATOCRIT     float64
HAEMOGLOBINS    float64
ERYTHROCYTE     float64
LEUCOCYTE       float64
THROMBOCYTE       int64
MCH             float64
MCHC            float64
MCV             float64
AGE               int64
SEX              object
SOURCE            int64
dtype: object

# **3. Let's process the data and define attributes**


1.   The prediction will be executed only to adult people (>=18).
2.   The other attributes must be positive and a number.
3. Remenber! We have numerical and categorical attributes.







In [ ]:
columns = data.columns
columns

Index(['HAEMATOCRIT', 'HAEMOGLOBINS', 'ERYTHROCYTE', 'LEUCOCYTE',
       'THROMBOCYTE', 'MCH', 'MCHC', 'MCV', 'AGE', 'SEX', 'SOURCE'],
      dtype='object')

Pay attention again on column AGE. We have only adults!

In [ ]:
columns_changed = ['HAEMATOCRIT', 'HAEMOGLOBINS', 'ERYTHROCYTE', 'LEUCOCYTE', 'THROMBOCYTE', 'MCH', 'MCHC', 'MCV', 'AGE', 'SOURCE']
for i in columns_changed:
  if i != 'AGE':
    data.drop(data[data[i] < 0].index, inplace=True)
  else:
    data.drop(data[data[i] < 18].index, inplace=True)
data.describe()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SOURCE
count,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000,2971.000000
mean,38.183945,12.740424,4.500074,8.834197,260.722316,28.475564,33.348435,85.318411,50.964322,0.401885
std,6.125176,2.139588,0.792586,5.036751,112.754816,2.636728,1.261048,6.695162,18.629294,0.490361
min,13.700000,3.800000,1.480000,1.100000,10.000000,14.900000,26.000000,54.000000,18.000000,0.000000
25%,34.100000,11.300000,3.980000,5.800000,195.000000,27.600000,32.700000,82.600000,34.000000,0.000000
50%,38.700000,12.900000,4.520000,7.700000,258.000000,28.900000,33.400000,85.900000,52.000000,0.000000
75%,42.600000,14.300000,5.010000,10.400000,322.000000,30.000000,34.100000,89.100000,66.000000,1.000000
max,69.000000,18.900000,7.860000,76.600000,1121.000000,40.800000,38.400000,115.600000,99.000000,1.000000


Here, we can notice the division of the dataset into: predictive attributes and targets.

In [ ]:
predictors = data.iloc[:, 0:10].values
targets = data.iloc[:, 10].values

SimpleImputer assigns the missing values ​​to the mean value of the column.

In [ ]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(predictors[:, 0:9])
predictors[:, 0:9] = imputer.transform(predictors[:, 0:9])
predictors

array([[33.8, 11.1, 4.18, ..., 80.9, 33.0, 'F'],
       [44.6, 14.0, 6.86, ..., 65.0, 36.0, 'M'],
       [42.9, 14.0, 4.57, ..., 93.9, 70.0, 'F'],
       ...,
       [38.2, 12.4, 4.4, ..., 86.8, 64.0, 'M'],
       [43.4, 15.2, 5.29, ..., 82.0, 33.0, 'M'],
       [32.4, 10.7, 3.66, ..., 88.5, 72.0, 'F']], dtype=object)

OneHotEncoder encodes categorical features as a one-hot numeric array.

In [ ]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [9])],remainder='passthrough')
predictors = onehotencorder.fit_transform(predictors)
predictors

array([[1.0, 0.0, 33.8, ..., 32.8, 80.9, 33.0],
       [0.0, 1.0, 44.6, ..., 31.4, 65.0, 36.0],
       [1.0, 0.0, 42.9, ..., 32.6, 93.9, 70.0],
       ...,
       [0.0, 1.0, 38.2, ..., 32.5, 86.8, 64.0],
       [0.0, 1.0, 43.4, ..., 35.0, 82.0, 33.0],
       [1.0, 0.0, 32.4, ..., 33.0, 88.5, 72.0]], dtype=object)

LabelEncoder converts each value in a column to a number.

In [ ]:
#It's not required
labelencorder_targets = LabelEncoder()
targets = labelencorder_targets.fit_transform(targets)
targets

array([1, 0, 0, ..., 0, 0, 1])

Standardize features by removing the mean and scaling to unit variance.

In [ ]:
scaler = StandardScaler()
predictors = scaler.fit_transform(predictors)
predictors

array([[ 1.05716845e+00, -1.05716845e+00, -7.15846024e-01, ...,
        -4.34977190e-01, -6.60052012e-01, -9.64467373e-01],
       [-9.45923046e-01,  9.45923046e-01,  1.04766547e+00, ...,
        -1.54535164e+00, -3.03530100e+00, -8.03403562e-01],
       [ 1.05716845e+00, -1.05716845e+00,  7.70075700e-01, ...,
        -5.93602111e-01,  1.28197546e+00,  1.02198629e+00],
       ...,
       [-9.45923046e-01,  9.45923046e-01,  2.62162317e-03, ...,
        -6.72914572e-01,  2.21329688e-01,  6.99858669e-01],
       [-9.45923046e-01,  9.45923046e-01,  8.51719751e-01, ...,
         1.30989694e+00, -4.95726611e-01, -9.64467373e-01],
       [ 1.05716845e+00, -1.05716845e+00, -9.44449366e-01, ...,
        -2.76352269e-01,  4.75287127e-01,  1.12936216e+00]])

# **4. Determination of 3 best algorithms**

In [ ]:
classifier = [LogisticRegression(),SVC(kernel = 'rbf', random_state = 1, C = 2.0, probability=True),KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2),RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0),MLPClassifier(verbose = False, max_iter = 1000, tol = 0.000010, solver='adam', hidden_layer_sizes=(100), activation = 'relu', batch_size=200, learning_rate_init=0.001)]
type_classifier = ['Logistic', 'SVC', 'KNN', 'RF', 'MLP']
size_classifier = len(classifier)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state = 0)

result_tp = []
mean_main_tp = []

result_fp = []
mean_main_fp = []

result_fn = []
mean_main_fn = []

result_tn = []
mean_main_tn = []

result_auc = []
mean_main_auc = []

result_accuracy = []
mean_main_accuracy = []

result_precision = []
mean_main_precision = []

result_recall = []
mean_main_recall = []

result_f1_score = []
mean_main_f1_score = []

for j in range(0,size_classifier):
  for index_trainning, index_test in kfold.split(predictors, np.zeros(shape=(targets.shape[0], 1))):
    classifier_model = classifier[j]
    classifier_model.fit(predictors[index_trainning], targets[index_trainning])
    prevision = classifier_model.predict_proba(predictors[index_test])
    prevision = prevision[:,1]

    auc_score = round(roc_auc_score(targets[index_test],prevision),3)
    result_auc.append(auc_score)

    prevision = classifier_model.predict(predictors[index_test])

    tp, fp, fn, tn = confusion_matrix(targets[index_test], prevision).ravel()
    result_tp.append(tp)
    result_fp.append(fp)
    result_fn.append(fn)
    result_tn.append(tn)

    accuracy = round(accuracy_score(targets[index_test],prevision),3)
    result_accuracy.append(accuracy)

    precision = round(precision_score(y_pred = prevision, y_true = targets[index_test]),3)
    result_precision.append(precision)

    recall = round(recall_score(y_pred = prevision, y_true = targets[index_test]),3)
    result_recall.append(recall)

    f1_score = round(2*precision*recall/(precision + recall),3)
    result_f1_score.append(f1_score)

  result_score = np.asarray(result_tp)
  mean_result = mean(result_score)
  mean_main_tp.append(mean_result)

  result_score = np.asarray(result_fp)
  mean_result = mean(result_score)
  mean_main_fp.append(mean_result)

  result_score = np.asarray(result_fn)
  mean_result = mean(result_score)
  mean_main_fn.append(mean_result)

  result_score = np.asarray(result_tn)
  mean_result = mean(result_score)
  mean_main_tn.append(mean_result)

  result_score = np.asarray(result_auc)
  mean_result = mean(result_score)
  mean_main_auc.append(mean_result)

  result_score = np.asarray(result_accuracy)
  mean_result = mean(result_score)
  mean_main_accuracy.append(mean_result)

  result_score = np.asarray(result_precision)
  mean_result = mean(result_score)
  mean_main_precision.append(mean_result)

  result_score = np.asarray(result_recall)
  mean_result = mean(result_score)
  mean_main_recall.append(mean_result)

  result_score = np.asarray(result_f1_score)
  mean_result = mean(result_score)
  mean_main_f1_score.append(mean_result)

  result_tp = []
  result_fp = []
  result_fn = []
  result_tn = []
  result_auc = []
  result_accuracy = []
  result_precision = []
  result_recall = []
  result_f1_score = []

print('Average tp,fp,fn,tn for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_tp[k]) + ', ' + str(mean_main_fp[k])+ ', ' + str(mean_main_fn[k])+ ', ' + str(mean_main_tn[k]))
print("-------------------------------------------------------------------------")
print('Average AUC score for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_auc[k]))
print("-------------------------------------------------------------------------")
print('Average accuracy score for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_accuracy[k]))
print("-------------------------------------------------------------------------")
print('Average precision for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_precision[k]))
print("-------------------------------------------------------------------------")
print('Average recall for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_recall[k]))
print("-------------------------------------------------------------------------")
print('Average f1 score for each algorithm:')
for k in range(0,size_classifier):
  print(str(type_classifier[k]) + ': ' + str(mean_main_f1_score[k]))


Average tp,fp,fn,tn for each algorithm:
Logistic: 154, 22, 60, 59
SVC: 156, 21, 51, 67
KNN: 143, 34, 47, 71
RF: 149, 28, 46, 73
MLP: 147, 30, 46, 72
-------------------------------------------------------------------------
Average AUC score for each algorithm:
Logistic: 0.7626
SVC: 0.8002
KNN: 0.7618
RF: 0.7974
MLP: 0.7942
-------------------------------------------------------------------------
Average accuracy score for each algorithm:
Logistic: 0.72
SVC: 0.7537
KNN: 0.7247
RF: 0.7473
MLP: 0.7424999999999999
-------------------------------------------------------------------------
Average precision for each algorithm:
Logistic: 0.7224
SVC: 0.7591
KNN: 0.6789
RF: 0.7175
MLP: 0.7086
-------------------------------------------------------------------------
Average recall for each algorithm:
Logistic: 0.4946
SVC: 0.5669
KNN: 0.5991
RF: 0.6113
MLP: 0.6098
-------------------------------------------------------------------------
Average f1 score for each algorithm:
Logistic: 0.5857
SVC: 0.

# **5. Save the 4 best algorithms**

In [ ]:
classifier = [SVC(kernel = 'rbf', random_state = 1, C = 2.0, probability=True),KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2),RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0),MLPClassifier(verbose = False, max_iter = 1000, tol = 0.000010, solver='adam', hidden_layer_sizes=(100), activation = 'relu', batch_size=200, learning_rate_init=0.001)]
type_classifier = ['SVC', 'KNN', 'RF', 'MLP']
size_classifier = len(classifier)
for l in range(0,size_classifier):
  classifier_model = classifier[l]
  classifier_model.fit(predictors, targets)
  pickle.dump(classifier_model, open(str(type_classifier[l]) + '.sav', 'wb'))

# **6. Combination of algorithms**

In [ ]:
new_register = [[1.0, 0.0, 30, 11, 4, 4, 150, 26, 32, 80, 30]]
new_register = np.asarray(new_register)
new_register = new_register.reshape(-1, 1)
scaler = StandardScaler()
new_register = scaler.fit_transform(new_register)
new_register = new_register.reshape(-1, 11)
new_register

array([[-0.75654952, -0.77986057, -0.08052908, -0.52343903, -0.68661637,
        -0.68661637,  2.71679689, -0.17377328, -0.03390698,  1.0850234 ,
        -0.08052908]])

In [ ]:
svm = pickle.load(open('SVC.sav', 'rb'))
knn = pickle.load(open('KNN.sav', 'rb'))
rf = pickle.load(open('RF.sav', 'rb'))
mlp = pickle.load(open('MLP.sav', 'rb'))

In [ ]:
predict_svm = svm.predict(new_register)
predict_knn = knn.predict(new_register)
predict_rf = rf.predict(new_register)
predict_mlp = mlp.predict(new_register)

In [ ]:
print(predict_knn,predict_svm,predict_rf,predict_mlp)

[0] [0] [0] [0]


In [ ]:
proba_svm = svm.predict_proba(new_register)
max_svm = proba_svm.max()

proba_knn = knn.predict_proba(new_register)
max_knn = proba_knn.max()

proba_rf = rf.predict_proba(new_register)
max_rf = proba_rf.max()

proba_mlp = mlp.predict_proba(new_register)
max_mlp = proba_mlp.max()

confidence = np.array([max_svm,max_knn,max_rf,max_mlp])
max_confidence = confidence.max()
print(confidence, max_confidence)

max_general = 0.8

[0.79848027 0.8        0.775      0.86486476] 0.8648647616905916


In [ ]:
in_care_patient = 0
out_care_patient = 0

if max_knn >= max_general:
  if predict_knn[0] == 1:
      in_care_patient += 1
  else:
      out_care_patient += 1

if max_svm >= max_general:
  if predict_svm[0] == 1:
      in_care_patient += 1
  else:
      out_care_patient += 1

if max_rf >= max_general:
  if predict_rf[0] == 1:
      in_care_patient += 1
  else:
      out_care_patient += 1

if max_mlp >= max_general:
  if predict_mlp[0] == 1:
      in_care_patient += 1
  else:
      out_care_patient += 1

print('Votes for Out care patient: ' + str(out_care_patient))
print('Votes for In care patient: ' + str(in_care_patient))
print("-------------------------------------------------------------------------")
if in_care_patient > out_care_patient:
  print('In care patient')
elif in_care_patient == out_care_patient:
  print('Result: Inconclusive result')
else:
  print('Result: Out care patient')
print('Maximum confidence:' + str(round(max_confidence,3)))

Votes for Out care patient: 2
Votes for In care patient: 0
-------------------------------------------------------------------------
Result: Out care patient
Maximum confidence:0.865
